In [1]:
import torch
from torch import tensor
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'
#torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is using")

training_data = datasets.MNIST("data", train=True, download=True, transform=ToTensor())
test_data = datasets.MNIST("data", train=False, download=True, transform=ToTensor())

trainLoader = torch.utils.data.DataLoader(training_data, batch_size=100, shuffle=False, drop_last=False)
testLoader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False, drop_last=True)


class network(nn.Module):  # 네트워크 모델 정의
    def __init__(self):
        super(network, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, (3, 3), padding=1),   # in_channels = 1, out_channels = 6, kernel_size = 3       output = 28 * 24 * 6
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                   # max 풀링  kernel_size = 2, stride = 2                     output = 14 * 14 * 6
            nn.Conv2d(6, 16, (3, 3), padding=1),  # in_channels = 6, out_channels = 16, kernel_size = 3      output = 14 * 14 * 16
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                    # max 풀링  kernel_size = 2, stride = 2                     output =  7 *  7 * 16
        )
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        # torch.nn.MaxPool2d( kernel_size , stride = None , padding = 0 , dilation = 1 , return_indices = False , ceil_mode = False )

        self.fully_connected = nn.Sequential(
            nn.Linear(16 * 7 * 7, 120),  # 1층 레이어
            nn.Linear(120, 84),          # 2층 레이어
            nn.Linear(84, 10),            # 3층 레이어
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 16 * 7 * 7)  # 1차원 전환  (nn.flatten)
        x = self.fully_connected(x)
        return x


def train():
    for epoch in range(Epochs):
        loss_sum = 0
        for data, target in trainLoader:
            X, y = data.to(device), target.to(device)  # cross
            optimizer.zero_grad()
            prediction = model(X)  # 결과 출력
            loss = criterion(prediction, y)  # cross 로스 계산
            loss.backward()  # 로스 역전파
            optimizer.step()  # 실질적 웨이트 수정
            loss_sum += loss.item()
        print("epoch = %d   loss = %f" % (epoch + 1, round(loss_sum / batch_count, 3)))
        test()


def test():
    correct = 0
    with torch.no_grad():
        for data, target in testLoader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)  # 출력 계산

            # 추론 계산
            _, predicted = torch.max(outputs, 1)  # 가장 큰 인덱스 위치를 리턴함  @ return value, index
            correct += predicted.eq(target).sum()  # 정답과 일치한 경우 정답 카운트를 증가

    data_num = len(test_data)  # 데이터 총 건수
    print("accuracy = {}/10000\n".format(correct))





cuda is using


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [2]:
model = network().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

Epochs = 2
batch_count = len(trainLoader)

train()

test()

epoch = 1   loss = 0.367000
accuracy = 9671/10000

epoch = 2   loss = 0.098000
accuracy = 9723/10000

accuracy = 9723/10000



In [ ]:
# 테스트 데이터를 사용한 모델 테스트.
with torch.no_grad(): # gradient 계산 중지
  for data, target in trainLoader:
    X_test, Y_test = data.to(device), target.to(device)  # cross
    prediction = model(X_test)  # 결과 출력
    print(target)


In [ ]:
  #prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

  # 무작위 예측
  r = random.randint(0, len(X_test) - 1)
  r = 99
  X_single_data = X_test[r:r + 1].view(-1, 28 * 28).float().to(device)
  Y_single_data = Y_test[r:r + 1].to(device)

  print('Label: ', Y_single_data.item())
  X_1 = X_single_data.view(1, 1, 28, 28)
  single_prediction = model(X_1)
  print('Prediction: ', torch.argmax(single_prediction, 1).item())

  plt.imshow(torch.Tensor.cpu(X_test[r:r + 1].view(28, 28)), cmap='Greys', interpolation='nearest')
  plt.show()

In [ ]:
### 2D display CPU version
X_test = test_data.test_data.float()
r = 9999

plt.imshow(X_test[r:r + 1].view(28, 28), cmap='Greys', interpolation='nearest')
plt.show()

In [ ]:
###  2D display GPU version
X_test = test_data.test_data.float().to(device)
r = 9999
plt.imshow(torch.Tensor.cpu(X_test[r:r + 1].view(28, 28)), cmap='Greys', interpolation='nearest')
plt.show()